In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline

pd.options.display.max_columns = 100

In [2]:
from secret import local_config

# Propub Specific Vote Data

In [3]:
### Set connection to sqlite DB
### Source: local_config db_params
conn = sqlite3.connect(local_config.db_params['database'])

### Establish datatables from DB tables
df_voters = pd.read_sql_query('SELECT * from {}'.format(
    local_config.db_params['table_individual_votes']), conn)
df_overview = pd.read_sql_query('SELECT * from {}'.format(
    local_config.db_params['table_overview']), conn)

# Close connection
conn.close()

# Remove Index
del df_overview['index']
del df_voters['index']

In [4]:
full = df_voters.merge(df_overview, on=['congress','session','roll_call'])

In [5]:
full['republican_total'] = full['republican_yes'] + full['republican_no'] + full['republican_not_voting']
full['democratic_total'] = full['democratic_yes'] + full['democratic_no'] + full['democratic_not_voting']
full['independent_total'] = full['independent_yes'] + full['independent_no'] + full['independent_not_voting']
full['total'] = full[['republican_total','democratic_total','independent_total']].sum(axis=1)
full['total_yes'] = full['republican_yes'] +full['democratic_yes'] +full['independent_yes']
full['republican_per_yes'] = full['republican_yes']/full['republican_total']
full['democratic_per_yes'] = full['democratic_yes']/full['democratic_total']
full['total_per_yes'] = full['total_yes']/full['total']

In [6]:
full['congress_id'] = full['congress'] + (full['session']-1)/2

In [7]:
"""The further from zero the value the more partisan the closer to 0 the less 
(more cross isle balance)"""

full['partisan_weight'] = (
    (full['democratic_per_yes']-full['republican_per_yes'])/
                    full['total_per_yes']
)

In [8]:
def fix_vote_type(vote_type):
    try:
        numer, denom = vote_type.split('/')
        perct = float(numer)/float(denom)*100
    except:
        perct = vote_type
    finally:
        if perct == 50:
            
            return 51
        else:
            return np.round(perct,0).astype(int)

full['vote_type'] = full['vote_type'].map(fix_vote_type)

In [9]:
full['yes_needed'] = (full['vote_type']/100)*full['total']
full['pass'] = np.where(full['total_yes']>full['yes_needed'],True,False)

In [10]:
full['vote_position'] = full['vote_position'].replace(
    'Yes', 1).replace(
    'No', -1).replace(
    'Not Voting', 0).replace(
    'Present', 0)

In [11]:
full['vote_partisonship'] = full['vote_position']*full['partisan_weight']

In [12]:
agg_method = {'vote_position':'sum',
              'democratic_no':'sum',
              'democratic_yes':'sum',
              'republican_no':'sum',
              'republican_yes':'sum',
              'independent_yes':'sum',
              'independent_no':'sum',
              'vote_partisonship':['mean','median','std'],
              'partisan_weight':'sum',
              'democratic_present':'count',
              'party':'first'}

# TF Analysis

In [13]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import re

In [14]:
def helper_slit(str_, delim, seg):
    try:
        str_list = str_.split(delim)
        return str_list[seg]
    except:
        return str_

In [15]:
def str_check(str_, sub_str, case=True):
    if type(str_) != str:
        return False
    if case == False:
        str_ = str_.lower()
        sub_str = sub_str.lower()
    if sub_str in str_:
        return True

In [16]:
df = df_overview.copy()

In [275]:
def search_bill(des):
    if type(des) != str:
        return 'No Bill'
    sen_str = 'S.[ 0-9]+'
    hou_str = 'Res.[ 0-9]+'
    amd_str = 'Amendment No[. ]+[0-9]+'
    amd_str2 = 'Amdt. [No]+[0-9]+'
    amd_str3 = 'Amdt[. ][0-9]+'
    hr_str = 'H.R. [0-9]+'
    no_str = 'No. [0-9]+'
    str_l = [sen_str, hou_str, amd_str, amd_str2, amd_str3, hr_str, no_str, '']
    str_n = 0
    fin = []
    while len(fin) < 1:
        str_f = str_l[str_n]
        fin = re.findall(pattern=str_f, string=des)
        str_n += 1
        if str_n == len(str_l):
            if 'nomination' in des.lower():
                return 'nomination'
            else:
                fin = ['No Bill']
    final = fin[0].replace('.A',
                   'A').lower().replace('.',
                   '. ').replace('  ',
                   ' ').replace('amdt.',
                   'amendment').replace('h. r.',
                   'h.r.')
    if final[0] == ' ':
        final = final[1:]
    if 'amendment no.' in final:
        final = 'amendment no. ' + final.split('amendment no.')[-1]
    return final

In [276]:
df['bill'] = df['description'].apply(search_bill)

In [277]:
df[df['bill'] == 'no bill']['bill'].value_counts()

no bill    4627
Name: bill, dtype: int64

In [ ]:
(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

# Shuffle the training set
order = np.argsort(np.random.random(train_labels.shape))
train_data = train_data[order]
train_labels = train_labels[order]